Постройте воронку, которая показывала бы в динамике по неделям и месяцам:
1.   сколько клиентов посещает наш сайт;
2.   какая доля из них (посетивших) регистрируется на нём/подаёт заявку на игру/приходит на игру (неважно, в каком месяце);
3.   какая доля из них (зарегестрировавшихся) подает заявку на игру (неважно, в каком месяце);
4.   какая доля из них (зарегестрировавшихся и подавших заявку на игру) по факту приходит на игру (неважно, в каком месяце).

Изобразите:

1.   Кол-во клиентов, которые посещают наш сайт / регистрируются на сайте / подают заявку на игру/ приходят на игру помесячно и понедельно. Добавьте легенду, подписи осей и название диаграммы.

Дополнительно сделайте табличку с долей каждой группы от общего числа посетителей для помесячного графика.

2.   Помесячно (и затем понедельно) на одной гистограмме
- кол-во клиентов, которые посетили сайт / зарегистрировались / подали заявку на игру / пришли на игру и долю зарегистрировавшихся от посетителей / долю подавших заявку от регистраций / долю пришедших хотя бы на 1 игру от подававших заявку (абсолюты и конверсии должны быть на разных осях соответственно)


Какие выводы можно сделать исходя из построенных конверсий, какие точки роста продукта вы видите?

// Всего можно получить 6 баллов
*   построена полная воронка с помощью sql-запроса - 2 балла
*   выполнен 1 пункт - 2 балла
*   выполнен 2 пункт - 2 балла



In [1]:
import psycopg2
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from plotly import graph_objects as go
from plotly.data import tips
import plotly.graph_objects as go
%matplotlib inline

conn = psycopg2.connect(
    host="158.160.110.228",
    port=5432,
    database="postgres",
    user="student",
    password="JvLda93aA",
)
curr=conn.cursor()

KeyboardInterrupt: 

In [ ]:
sql_months = '''
select
	(to_char(visit_dttm, 'YYYY ') || (to_char(visit_dttm, 'Month'))) as year_month,
	date_trunc('month', visit_dttm)::date as first_day_of_month,
	count(distinct c.client_rk) as visitors,
	count(distinct a.account_rk) as registered,
	count(distinct ap.account_rk) as applied,
	count(distinct ap.account_rk) filter (
where
	game_flg = 1) as played
from
		msu_analytics.client c
left join msu_analytics.account a
		using(client_rk)
left join msu_analytics.application ap
		using(account_rk)
left join msu_analytics.game g
		using(game_rk)
group by
	1,
	2
order by
	2
'''
sql_weeks = '''
select
	date_trunc('week', visit_dttm)::date as first_day_of_week,
	count(distinct c.client_rk) as visitors,
	count(distinct a.account_rk) as registered,
	count(distinct ap.account_rk) as applied,
	count(distinct ap.account_rk) filter (
where
	game_flg = 1) as played
from
		msu_analytics.client c
left join msu_analytics.account a
		using(client_rk)
left join msu_analytics.application ap
		using(account_rk)
left join msu_analytics.game g
		using(game_rk)
group by
	1
order by
	1
'''

In [ ]:
#Табличка по месяцам
df_months = pd.read_sql(sql_months, conn)
df_months.head(10)

C:\Users\Huawei\AppData\Local\Temp\ipykernel_5900\1252440817.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_months = pd.read_sql(sql_months, conn)


,year_month,first_day_of_month,visitors,registered,applied,played
0,2022 September,2022-09-01,778,190,51,39
1,2022 October,2022-10-01,843,217,65,56
2,2022 November,2022-11-01,786,200,56,50
3,2022 December,2022-12-01,805,196,57,33
4,2023 January,2023-01-01,852,213,64,20
5,2023 February,2023-02-01,32,8,0,0


In [ ]:
#Табличка по неделям
df_weeks = pd.read_sql(sql_weeks, conn)
df_weeks.head(10)

C:\Users\Huawei\AppData\Local\Temp\ipykernel_5900\723851140.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_weeks = pd.read_sql(sql_weeks, conn)


,first_day_of_week,visitors,registered,applied,played
0,2022-08-29,111,32,7,5
1,2022-09-05,184,31,6,5
2,2022-09-12,167,43,16,8
3,2022-09-19,181,45,12,11
4,2022-09-26,195,59,17,17
5,2022-10-03,204,58,18,15
6,2022-10-10,190,44,13,13
7,2022-10-17,171,41,12,10
8,2022-10-24,189,48,15,11
9,2022-10-31,197,47,16,14


In [ ]:
#считаю конверсию регистрации
df_months['conversion_reg']=df_months['registered']/df_months['visitors']*100

#считаю конверсию подавших на игру (долю подавших от зареганых)
df_months['conversion_app']=df_months['applied']/df_months['registered']*100

#считаю конверсию подавших на игру (долю подавших от зареганых)
df_months['conversion_game']=df_months['played']/df_months['applied']*100

df_months.head(10)

,year_month,first_day_of_month,visitors,registered,applied,played,conversion_reg,conversion_app,conversion_game
0,2022 September,2022-09-01,778,190,51,39,24.421594,26.842105,76.470588
1,2022 October,2022-10-01,843,217,65,56,25.741400,29.953917,86.153846
2,2022 November,2022-11-01,786,200,56,50,25.445293,28.000000,89.285714
3,2022 December,2022-12-01,805,196,57,33,24.347826,29.081633,57.894737
4,2023 January,2023-01-01,852,213,64,20,25.000000,30.046948,31.250000
5,2023 February,2023-02-01,32,8,0,0,25.000000,0.000000,NaN


In [ ]:
#строю график по общему количеству

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_months['year_month'],
    y=df_months['visitors'],
    name="Посещения сайта",
    marker_color='red',
    mode='lines+markers+text',
    text=df_months['visitors'],
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=df_months['year_month'],
    y=df_months['registered'],
    name="Регистрации",
    marker_color='blue',
    mode='lines+markers+text',
    text=df_months['registered'],
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=df_months['year_month'],
    y=df_months['applied'],
    name="Подали заявку на игру",
    marker_color='purple',
    text=df_months['applied']
))

fig.add_trace(go.Scatter(
    x=df_months['year_month'],
    y=df_months['played'],
    mode='lines+markers', 
    name='Пришли на игру',
    marker_color='yellow',
    text=df_months['applied']
))

fig.update_layout(
    legend=dict(orientation="h"),
    title='Общее количество по месяцам',
    yaxis=dict(
        title=dict(text="Количество"),
        side="left",
        range=[0, 1000]
    )
)

fig.show()

In [ ]:
#строю график 

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_months['year_month'],
    y=df_months['conversion_reg'],
    name="Доля тех кто зарегистрировался на сайте из посетивших, %",
    marker_color='tan',
    text=df_months['conversion_reg'].round(2)
))

fig.add_trace(go.Bar(
    x=df_months['year_month'],
    y=df_months['conversion_app'],
    name="Доля тех кто подал на игру из зарегистрированных, %",
    marker_color='navajowhite',
    text=df_months['conversion_app'].round(2)
))

fig.add_trace(go.Bar(
    x=df_months['year_month'],
    y=df_months['conversion_game'],
    name="Доля тех кто пришел на игру из подавших, %",
    marker_color='blanchedalmond',
    text=df_months['conversion_game'].round(2)
))

fig.update_layout(
    legend=dict(orientation="h"),
    title='Конверсии по месяцам',
    yaxis=dict(
        title=dict(text="Конверсия, %"),
        side="left",
        range=[0, 100]
    ),
    yaxis2=dict(
        title=dict(text="Посещения сайта"),
        side="right",
        range=[0, 900],
        overlaying="y"
    )
)

fig.show()

In [ ]:
#а теперь все на одной картинке (не знаю зачем это нужно)


fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_months['year_month'],
    y=df_months['conversion_reg'],
    name="Доля тех кто зарегистрировался на сайте из посетивших, %",
    marker_color='tan',
    text=df_months['conversion_reg'].round(2)
))

fig.add_trace(go.Bar(
    x=df_months['year_month'],
    y=df_months['conversion_app'],
    name="Доля тех кто подал на игру из зарегистрированных, %",
    marker_color='navajowhite',
    text=df_months['conversion_app'].round(2)
))

fig.add_trace(go.Bar(
    x=df_months['year_month'],
    y=df_months['conversion_game'],
    name="Доля тех кто пришел на игру из подавших, %",
    marker_color='blanchedalmond',
    text=df_months['conversion_game'].round(2)
))

fig.add_trace(go.Scatter(
    x=df_months['year_month'],
    y=df_months['visitors'],
    mode='lines+markers+text',
    marker_color='red', 
    name='Количество посещений сайта',
    textposition="top center",
    yaxis='y2'
))


fig.add_trace(go.Scatter(
    x=df_months['year_month'],
    y=df_months['registered'],
    yaxis="y2",
    name="Регистрации",
    marker_color='blue',
    text=df_months['registered']
))

fig.add_trace(go.Scatter(
    x=df_months['year_month'],
    y=df_months['applied'],
    yaxis="y2",
    name="Подали заявку на игру",
    marker_color='purple',
    text=df_months['applied']
))

fig.add_trace(go.Scatter(
    x=df_months['year_month'],
    y=df_months['played'],
    yaxis="y2",
    mode='lines+markers', 
    name='Пришли на игру',
    marker_color='yellow',
    text=df_months['played']
))

fig.update_layout(
    legend=dict(orientation="h"),
    title='Общий результат по месяцам',
    yaxis=dict(
        title=dict(text="Конверсия, %"),
        side="left",
        range=[0, 100]
    ),
    yaxis2=dict(
        title=dict(text="Количество"),
        side="right",
        range=[0, 900],
        overlaying="y"
    )
)

fig.show()

In [ ]:
#считаю конверсию регистрации
df_weeks['conversion_reg']=df_weeks['registered']/df_weeks['visitors']*100

#считаю конверсию подавших на игру (долю подавших от зареганых)
df_weeks['conversion_app']=df_weeks['applied']/df_weeks['registered']*100

#считаю конверсию подавших на игру (долю подавших от зареганых)
df_weeks['conversion_game']=df_weeks['played']/df_weeks['applied']*100

df_weeks.head(10)

,first_day_of_week,visitors,registered,applied,played,conversion_reg,conversion_app,conversion_game
0,2022-08-29,111,32,7,5,28.828829,21.875000,71.428571
1,2022-09-05,184,31,6,5,16.847826,19.354839,83.333333
2,2022-09-12,167,43,16,8,25.748503,37.209302,50.000000
3,2022-09-19,181,45,12,11,24.861878,26.666667,91.666667
4,2022-09-26,195,59,17,17,30.256410,28.813559,100.000000
5,2022-10-03,204,58,18,15,28.431373,31.034483,83.333333
6,2022-10-10,190,44,13,13,23.157895,29.545455,100.000000
7,2022-10-17,171,41,12,10,23.976608,29.268293,83.333333
8,2022-10-24,189,48,15,11,25.396825,31.250000,73.333333
9,2022-10-31,197,47,16,14,23.857868,34.042553,87.500000


In [ ]:
#строю график по общему количеству

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['visitors'],
    name="Посещения сайта",
    marker_color='red',
    mode='lines+markers+text',
    text=df_weeks['visitors'],
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['registered'],
    name="Регистрации",
    marker_color='blue',
    text=df_weeks['registered'],
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['applied'],
    name="Подали заявку на игру",
    marker_color='purple',
    text=df_weeks['applied']
))

fig.add_trace(go.Scatter(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['played'],
    mode='lines+markers', 
    name='Пришли на игру',
    text=df_weeks['applied']
))

fig.update_layout(
    legend=dict(orientation="h"),
    title='Общее количество по неделям',
    yaxis=dict(
        title=dict(text="Количество"),
        side="left",
        range=[0, 250]
    )
)

fig.show()

In [ ]:
#строю график 

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['conversion_reg'],
    name="Доля тех кто зарегистрировался на сайте из посетивших, %",
    marker_color='tan',
    text=df_weeks['conversion_reg'].round(2)
))

fig.add_trace(go.Bar(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['conversion_app'],
    name="Доля тех кто подал на игру из зарегистрированных, %",
    marker_color='navajowhite',
    text=df_weeks['conversion_app'].round(2)
))

fig.add_trace(go.Bar(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['conversion_game'],
    name="Доля тех кто пришел на игру из подавших, %",
    marker_color='blanchedalmond',
    text=df_weeks['conversion_game'].round(2)
))

fig.add_trace(go.Scatter(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['visitors'],
    mode='lines+markers', 
    name='Количество посещений сайта',
    yaxis='y2'
))

fig.add_trace(go.Scatter(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['registered'],
    yaxis="y2",
    name="Регистрации",
    marker_color='blue',
    text=df_weeks['registered']
))

fig.add_trace(go.Scatter(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['applied'],
    yaxis="y2",
    name="Подали заявку на игру",
    marker_color='purple',
    text=df_weeks['applied']
))

fig.add_trace(go.Scatter(
    x=df_weeks['first_day_of_week'],
    y=df_weeks['played'],
    yaxis="y2",
    mode='lines+markers', 
    name='Пришли на игру',
    marker_color='yellow',
    text=df_weeks['played']
))

fig.update_layout(
    legend=dict(orientation="h"),
    title='Конверсии по неделям',
    yaxis=dict(
        title=dict(text="Конверсия, %"),
        side="left",
        range=[0, 100]
    ),
    yaxis2=dict(
        title=dict(text="Посещения сайта"),
        side="right",
        range=[0, 300],
        overlaying="y"
    )
)

fig.show()

Выводы:

1. Сложно сделать какие-то выводы за Февраль 2023 года из-за нехватки данных.
2. Как мы видим конверсия регистрации на сайте по месяцам составляет 25%. То есть около 75% людей посетивших сайт не стали на нем регистрироваться. Возможно это связано с неудобностью интерфейса или формой для регистрации, которую люди по какой-то причине не хотят заполнять (например, не хотят оставлять личные данные).
3. Доля подавших на игру из зарегистрировавшихся составляет 30%, скорее всего это из-за того, что люди просто забывают о своей регистрации.
4. По какой-то причине в декабре 2022 резко подает доля людей, которые пришли на игру из подавших.

Точки роста:

1. Для тех кто регистрируется, но не подает на игру, можно организовать рассылку с уведомлениями на почту или в виде смс на телефон.
2. Для увеличения конверсии регистрции можно упростить форму регистрации и запрашивать меньше личных данных у пользователей.